In [1]:
import findgaps
import generate_randomip
import gurobiutils
import gymenv_v2
# import gymenv
import solverutils
import testgymenv

In [2]:
import tensorflow as tf
import numpy as np
import random
from gymenv_v2 import make_multiple_env
import wandb
wandb.login()
run=wandb.init(project="finalproject", entity="ieor-4575", tags=["training-easy"])
#run=wandb.init(project="finalproject", entity="ieor-4575", tags=["training-hard"])
#run=wandb.init(project="finalproject", entity="ieor-4575", tags=["test"])

wandb: Currently logged in as: ieor-4575 (use `wandb login --relogin` to force relogin)
/opt/anaconda3/envs/ieor4575/lib/python3.6/site-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated since IPython 4.0. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


In [3]:
# Setup: You may generate your own instances on which you train the cutting agent.
custom_config = {
    "load_dir"        : 'instances/randomip_n60_m60',   # this is the location of the randomly generated instances (you may specify a different directory)
    "idx_list"        : list(range(20)),                # take the first 20 instances from the directory
    "timelimit"       : 50,                             # the maximum horizon length is 50
    "reward_type"     : 'obj'                           # DO NOT CHANGE reward_type
}

# Easy Setup: Use the following environment settings. We will evaluate your agent with the same easy config below:
easy_config = {
    "load_dir"        : 'instances/train_10_n60_m60',
    "idx_list"        : list(range(10)),
    "timelimit"       : 50,
    "reward_type"     : 'obj'
}

# Hard Setup: Use the following environment settings. We will evaluate your agent with the same hard config below:
hard_config = {
    "load_dir"        : 'instances/train_100_n60_m60',
    "idx_list"        : list(range(99)),
    "timelimit"       : 50,
    "reward_type"     : 'obj'
}

In [4]:
def calS(hg):
    h = hg[0]
    g = hg[1]
    p = tf.linalg.matmul(g,tf.transpose(h))
    p = tf.reduce_mean(p,axis=1)
    p = tf.linalg.normalize(p, ord=1)
    p = tf.nn.softmax(p[0])
    return p

In [5]:
def discounted_rewards(r, gamma):
    discounted_r = np.zeros_like(r)
    running_sum = 0
    for i in reversed(range(0,len(r))):
        discounted_r[i] = running_sum * gamma + r[i]
        running_sum = discounted_r[i]
    return list(discounted_r)

In [6]:
class Policy(object):
    
    def __init__(self, lr):

        self.model = tf.keras.models.Sequential([
                    tf.keras.layers.Input(shape=(1, 61)),
#                     tf.keras.layers.Dense(64, activation = "tanh"),
                    tf.keras.layers.LSTM(32, activation = "tanh"),
                    tf.keras.layers.Dense(10)
                ])

        self.optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
        
    
    def compute_prob(self, s):
        C = np.concatenate((s[0],np.array([s[1]]).T),axis=1)
        D = np.concatenate((s[3],np.array([s[4]]).T),axis=1)
        C = C.reshape(C.shape[0],1,C.shape[1])
        D = D.reshape(D.shape[0],1,D.shape[1])
        h = self.model(C)
        g = self.model(D)
        prob = tf.keras.layers.Lambda(calS)([h, g])
        prob = tf.cast(prob, tf.double)
        return prob.numpy()
    
    def train(self, states, actions, Qs):

        with tf.GradientTape() as tape:
            
            prob_selected = []

            for i in range(len(states)):
                C = np.concatenate((states[i][0],np.array([states[i][1]]).T),axis=1)
                D = np.concatenate((states[i][3],np.array([states[i][4]]).T),axis=1)
                C = C.reshape(C.shape[0],1,C.shape[1])
                D = D.reshape(D.shape[0],1,D.shape[1])
                h = self.model(C)
                g = self.model(D)
                prob = tf.keras.layers.Lambda(calS)([h, g])
                prob = tf.cast(prob, tf.double)

                prob_selected += [prob[actions[i]]]

            prob_selected = tf.convert_to_tensor(prob_selected)

            prob_selected += 1e-8

            loss = -tf.reduce_mean(Qs * tf.math.log(prob_selected))

            gradients = tape.gradient(loss, self.model.trainable_variables)  

            self.optimizer.apply_gradients(zip(gradients, self.model.trainable_variables))
            
        return loss.numpy()

In [7]:
env = make_multiple_env(**easy_config) 

loading training instances, dir instances/train_10_n60_m60 idx 0
loading training instances, dir instances/train_10_n60_m60 idx 1
loading training instances, dir instances/train_10_n60_m60 idx 2
loading training instances, dir instances/train_10_n60_m60 idx 3
loading training instances, dir instances/train_10_n60_m60 idx 4
loading training instances, dir instances/train_10_n60_m60 idx 5
loading training instances, dir instances/train_10_n60_m60 idx 6
loading training instances, dir instances/train_10_n60_m60 idx 7
loading training instances, dir instances/train_10_n60_m60 idx 8
loading training instances, dir instances/train_10_n60_m60 idx 9


In [8]:
lr = 1e-2  
numtrajs = 5 
gamma = .99
sigma = 0.02
epsilon = 0.6
iterations = 40
actor = Policy(lr) 
actor.model.build()

In [9]:
rrecord = []

for e in range(iterations):
    OBS = [] 
    ACTS = [] 
    ADS = []  
    VAL = []
    
    vas = actor.model.trainable_variables # set trainable_vars same for every iteration
    grashape = np.random.multivariate_normal(np.zeros(len(vas)), np.identity(len(vas)), 1).T 
    gradients = np.zeros_like(grashape)
    t = 0
    
    for num in range(numtrajs):

        s = env.reset() # get new IP
        
        done = False
        obss = []  # states
        acts = []   # actions
        rs = []
        repisode = 0
        counter = 0
        while not done:
            
            prob = actor.compute_prob(s)

            rand1 = random.random()
            if rand1>epsilon:
                action = list(np.random.randint(0, s[-1].size, 1))  
            else:
                action = [int(np.argmax(actor.compute_prob(s)))]
            
            acts += [action]
            obss += [s]
            
            s, r, done, _ = env.step(action)
            done_ = 1 if done else 0
            rs += [r]
            
            repisode += r
            print("ite",e,"traj",num,"step",counter, "action", action, "argmax", int(np.argmax(actor.compute_prob(s))), repisode)
            t += 1
            counter += 1
            
        VAL += discounted_rewards(rs, gamma)
        OBS += obss
        ACTS += acts
            
        rrecord += [repisode]
        fixedWindow=20
        movingAverage=0
        if len(rrecord) >= fixedWindow:
            movingAverage=np.mean(rrecord[len(rrecord)-fixedWindow:len(rrecord)-1])
        wandb.log({ "Training reward" : rrecord[-1], "training reward moving average" : movingAverage})


    actor.train(OBS,ACTS,np.array(VAL))


Using license file /Users/lan/gurobi.lic
ite 0 traj 0 step 0 action [36] argmax 1 0.0003492276023280283
ite 0 traj 0 step 1 action [1] argmax 1 0.0006329453194666712
ite 0 traj 0 step 2 action [1] argmax 1 0.0009394214575877413
ite 0 traj 0 step 3 action [58] argmax 1 0.0012637171284950455
ite 0 traj 0 step 4 action [1] argmax 1 0.01572065219443175
ite 0 traj 0 step 5 action [1] argmax 1 0.052210136439498456
ite 0 traj 0 step 6 action [1] argmax 1 0.05538015366573745
ite 0 traj 0 step 7 action [50] argmax 1 0.05561895482696855
ite 0 traj 0 step 8 action [1] argmax 1 0.06014018590030901
ite 0 traj 0 step 9 action [57] argmax 1 0.060874226114265184
ite 0 traj 0 step 10 action [38] argmax 1 0.06234596262993364
ite 0 traj 0 step 11 action [45] argmax 1 0.06649476410075295
ite 0 traj 0 step 12 action [4] argmax 1 0.06717708095402486
ite 0 traj 0 step 13 action [45] argmax 1 0.07662485670380192
ite 0 traj 0 step 14 action [21] argmax 1 0.08060318980460579
ite 0 traj 0 step 15 action [1] argm

ite 0 traj 2 step 34 action [87] argmax 0 0.11534908185194581
ite 0 traj 2 step 35 action [88] argmax 0 0.11534908185240056
ite 0 traj 2 step 36 action [0] argmax 0 0.1153490818528553
ite 0 traj 2 step 37 action [0] argmax 0 0.1153490818528553
ite 0 traj 2 step 38 action [0] argmax 0 0.1153490818537648
ite 0 traj 2 step 39 action [11] argmax 0 0.11534908185421955
ite 0 traj 2 step 40 action [71] argmax 0 0.1153490818546743
ite 0 traj 2 step 41 action [0] argmax 0 0.11534908185512904
ite 0 traj 2 step 42 action [0] argmax 0 0.11534908185512904
ite 0 traj 2 step 43 action [10] argmax 0 0.11534908185512904
ite 0 traj 2 step 44 action [0] argmax 0 0.11534908185558379
ite 0 traj 2 step 45 action [0] argmax 0 0.11534908185603854
ite 0 traj 2 step 46 action [0] argmax 0 0.11534908185649329
ite 0 traj 2 step 47 action [0] argmax 0 0.11534908185694803
ite 0 traj 2 step 48 action [42] argmax 0 0.11534908185740278
ite 0 traj 2 step 49 action [0] argmax 0 0.11534908185785753
ite 0 traj 3 step 0 ac

ite 1 traj 0 step 18 action [1] argmax 1 0.11023152771258538
ite 1 traj 0 step 19 action [1] argmax 1 0.11028225744485098
ite 1 traj 0 step 20 action [2] argmax 1 0.11030185903064194
ite 1 traj 0 step 21 action [28] argmax 1 0.11045694037102294
ite 1 traj 0 step 22 action [1] argmax 1 0.11062026386662183
ite 1 traj 0 step 23 action [1] argmax 1 0.11070663425471139
ite 1 traj 0 step 24 action [71] argmax 1 0.11103264286430203
ite 1 traj 0 step 25 action [1] argmax 1 0.11104281345888012
ite 1 traj 0 step 26 action [1] argmax 1 0.1110483865650167
ite 1 traj 0 step 27 action [71] argmax 1 0.11107150446605374
ite 1 traj 0 step 28 action [1] argmax 1 0.11107977948176995
ite 1 traj 0 step 29 action [55] argmax 1 0.11109359614147252
ite 1 traj 0 step 30 action [1] argmax 1 0.11110514617030276
ite 1 traj 0 step 31 action [1] argmax 1 0.11111177033717468
ite 1 traj 0 step 32 action [50] argmax 1 0.11111597956278274
ite 1 traj 0 step 33 action [1] argmax 1 0.11111818290191877
ite 1 traj 0 step 34

ite 1 traj 3 step 3 action [0] argmax 0 0.10131109041594755
ite 1 traj 3 step 4 action [16] argmax 0 0.1013110904164023
ite 1 traj 3 step 5 action [58] argmax 0 0.10131109041685704
ite 1 traj 3 step 6 action [35] argmax 0 0.10131109041731179
ite 1 traj 3 step 7 action [22] argmax 0 0.10131109041822128
ite 1 traj 3 step 8 action [26] argmax 0 0.10131109041822128
ite 1 traj 3 step 9 action [7] argmax 0 0.10131109041913078
ite 1 traj 3 step 10 action [0] argmax 0 0.10131109041958553
ite 1 traj 3 step 11 action [0] argmax 0 0.10131109041958553
ite 1 traj 3 step 12 action [55] argmax 0 0.10131109042004027
ite 1 traj 3 step 13 action [48] argmax 0 0.10131109042049502
ite 1 traj 3 step 14 action [0] argmax 0 0.10131109042049502
ite 1 traj 3 step 15 action [0] argmax 0 0.10131109042140451
ite 1 traj 3 step 16 action [0] argmax 0 0.10131109042231401
ite 1 traj 3 step 17 action [0] argmax 0 0.10131109042276876
ite 1 traj 3 step 18 action [35] argmax 0 0.10637035270974593
ite 1 traj 3 step 19 act

ite 2 traj 0 step 39 action [0] argmax 0 0.60724780731789
ite 2 traj 0 step 40 action [0] argmax 0 0.6072478073185721
ite 2 traj 0 step 41 action [0] argmax 0 0.6072478073194816
ite 2 traj 0 step 42 action [70] argmax 0 0.6072478073206184
ite 2 traj 0 step 43 action [33] argmax 0 0.6072478073213006
ite 2 traj 0 step 44 action [0] argmax 0 0.6072478073215279
ite 2 traj 0 step 45 action [0] argmax 0 0.6072478073217553
ite 2 traj 0 step 46 action [90] argmax 0 0.6072478073217553
ite 2 traj 0 step 47 action [48] argmax 0 0.6072478073217553
ite 2 traj 0 step 48 action [27] argmax 0 0.6072478073222101
ite 2 traj 0 step 49 action [0] argmax 0 0.6072478073283492
ite 2 traj 1 step 0 action [0] argmax 0 0.7082653196207502
ite 2 traj 1 step 1 action [0] argmax 0 0.7082653196221145
ite 2 traj 1 step 2 action [0] argmax 0 0.7082653196225692
ite 2 traj 1 step 3 action [60] argmax 0 0.7082653196225692
ite 2 traj 1 step 4 action [0] argmax 0 0.7082653196225692
ite 2 traj 1 step 5 action [0] argmax 0 0

...

ite 37 traj 3 step 40 action [0] argmax 0 0.7082653196343927
ite 37 traj 3 step 41 action [16] argmax 0 0.7082653196343927
ite 37 traj 3 step 42 action [0] argmax 0 0.7082653196343927
ite 37 traj 3 step 43 action [67] argmax 0 0.7082653196343927
ite 37 traj 3 step 44 action [0] argmax 0 0.7082653196348474
ite 37 traj 3 step 45 action [0] argmax 0 0.7082653196353021
ite 37 traj 3 step 46 action [0] argmax 0 0.7082653196357569
ite 37 traj 3 step 47 action [42] argmax 0 0.7082653196362116
ite 37 traj 3 step 48 action [0] argmax 0 0.7082653196362116
ite 37 traj 3 step 49 action [11] argmax 0 0.7082653196366664
ite 37 traj 4 step 0 action [0] argmax 0 0.7022608541797126
ite 37 traj 4 step 1 action [0] argmax 0 0.7022608541801674
ite 37 traj 4 step 2 action [0] argmax 0 0.7022608541810769
ite 37 traj 4 step 3 action [0] argmax 0 0.7022608541810769
ite 37 traj 4 step 4 action [0] argmax 0 0.7022608541810769
ite 37 traj 4 step 5 action [0] argmax 0 0.7022608541810769
ite 37 traj 4 step 6 actio

ite 38 traj 1 step 24 action [0] argmax 0 0.11534908186058601
ite 38 traj 1 step 25 action [0] argmax 0 0.11534908186104076
ite 38 traj 1 step 26 action [10] argmax 0 0.11534908186195025
ite 38 traj 1 step 27 action [36] argmax 0 0.115349081862405
ite 38 traj 1 step 28 action [0] argmax 0 0.11534908186285975
ite 38 traj 1 step 29 action [0] argmax 0 0.1153490818633145
ite 38 traj 1 step 30 action [0] argmax 0 0.11534908186376924
ite 38 traj 1 step 31 action [72] argmax 0 0.11534908186422399
ite 38 traj 1 step 32 action [39] argmax 0 0.11534908186467874
ite 38 traj 1 step 33 action [0] argmax 0 0.11534908186513348
ite 38 traj 1 step 34 action [0] argmax 0 0.11534908186513348
ite 38 traj 1 step 35 action [0] argmax 0 0.11534908186558823
ite 38 traj 1 step 36 action [0] argmax 0 0.11534908186558823
ite 38 traj 1 step 37 action [0] argmax 0 0.11534908186831672
ite 38 traj 1 step 38 action [64] argmax 0 0.11534908187059045
ite 38 traj 1 step 39 action [82] argmax 0 0.11534908187059045
ite 3

ite 38 traj 4 step 8 action [0] argmax 0 0.10131109041731179
ite 38 traj 4 step 9 action [15] argmax 0 0.10131109041776654
ite 38 traj 4 step 10 action [0] argmax 0 0.10131109041776654
ite 38 traj 4 step 11 action [28] argmax 0 0.10131109041822128
ite 38 traj 4 step 12 action [0] argmax 0 0.10131109041867603
ite 38 traj 4 step 13 action [0] argmax 0 0.10131109041867603
ite 38 traj 4 step 14 action [17] argmax 0 0.10131109041867603
ite 38 traj 4 step 15 action [0] argmax 0 0.10131109041867603
ite 38 traj 4 step 16 action [0] argmax 0 0.10131109041913078
ite 38 traj 4 step 17 action [12] argmax 0 0.10131109041958553
ite 38 traj 4 step 18 action [0] argmax 0 0.10131109042004027
ite 38 traj 4 step 19 action [0] argmax 0 0.10131109042049502
ite 38 traj 4 step 20 action [0] argmax 0 0.10131109042049502
ite 38 traj 4 step 21 action [4] argmax 0 0.10131109042094977
ite 38 traj 4 step 22 action [0] argmax 0 0.10188203517327565
ite 38 traj 4 step 23 action [55] argmax 0 0.10543274512383505
ite 3

ite 39 traj 1 step 42 action [0] argmax 0 0.7022608541965383
ite 39 traj 1 step 43 action [0] argmax 0 0.702260854196993
ite 39 traj 1 step 44 action [66] argmax 0 0.702260854196993
ite 39 traj 1 step 45 action [27] argmax 0 0.7022608541979025
ite 39 traj 1 step 46 action [0] argmax 0 0.702260854198812
ite 39 traj 1 step 47 action [0] argmax 0 0.702260854198812
ite 39 traj 1 step 48 action [0] argmax 0 0.7022608541997215
ite 39 traj 1 step 49 action [51] argmax 0 0.7022608541997215
ite 39 traj 2 step 0 action [0] argmax 0 0.8867413289726755
ite 39 traj 2 step 1 action [43] argmax 0 0.8867413289726755
ite 39 traj 2 step 2 action [11] argmax 0 0.8867413289731303
ite 39 traj 2 step 3 action [0] argmax 0 0.886741328973585
ite 39 traj 2 step 4 action [0] argmax 0 0.8867413289744945
ite 39 traj 2 step 5 action [0] argmax 0 0.8867413289763135
ite 39 traj 2 step 6 action [0] argmax 0 0.886741328977223
ite 39 traj 2 step 7 action [6] argmax 0 0.886741328977223
ite 39 traj 2 step 8 action [40] a

ite 39 traj 4 step 27 action [0] argmax 0 0.6072478073110688
ite 39 traj 4 step 28 action [0] argmax 0 0.6072478073115235
ite 39 traj 4 step 29 action [0] argmax 0 0.6072478073119782
ite 39 traj 4 step 30 action [87] argmax 0 0.6072478073119782
ite 39 traj 4 step 31 action [0] argmax 0 0.6072478073122056
ite 39 traj 4 step 32 action [0] argmax 0 0.607247807312433
ite 39 traj 4 step 33 action [0] argmax 0 0.6072478073128877
ite 39 traj 4 step 34 action [76] argmax 0 0.6072478073133425
ite 39 traj 4 step 35 action [0] argmax 0 0.6072478073140246
ite 39 traj 4 step 36 action [0] argmax 0 0.607247807314252
ite 39 traj 4 step 37 action [0] argmax 0 0.6072478073144794
ite 39 traj 4 step 38 action [0] argmax 0 0.6072478073147067
ite 39 traj 4 step 39 action [0] argmax 0 0.6072478073149341
ite 39 traj 4 step 40 action [51] argmax 0 0.6072478073151615
ite 39 traj 4 step 41 action [0] argmax 0 0.6072478073151615
ite 39 traj 4 step 42 action [0] argmax 0 0.6072478073151615
ite 39 traj 4 step 43 a

In [10]:
run.finish()

Training reward,0.60725
training reward moving average,0.75529
_runtime,7380
_timestamp,1619195688
_step,199


Training reward,▂▁▅▅▁█▅▄█▅▄▅▁▄▆▇▅▅▁▄▅▆█▁▅▄▁██▆▄▁▄▆██▄▅▅▄
training reward moving average,▁▁▁▁▆▇▇▇████▇▇▇▇▇████▇█▇▇█▇▇▇▇██▇▇▇███▇█
_runtime,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇███
_timestamp,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇███
_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
